# Estimating ECM Parameters from Multi-Pulse HPPC Data

This notebook provides example usage for estimating stationary parameters for a two RC branch Thevenin model using multi-pulse HPPC data.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP from its development branch and upgrade some dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets openpyxl pandas -q
%pip install pybop -q

import pandas as pd
import plotly.graph_objects as go
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.
/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


In this example, we use the default parameter value for the "Open-circuit voltage [V] as provided by the original PyBaMM class. The other relevant parameters for the ECM model implementation are updated as per the cell specification. First, we define the model class with two RC elements. One important thing here to note is "maximum solver timestep" (dt_max) needs to be set correctly to have a good fit.

In [ ]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})

# Load the parameters
parameter_values = pybamm.ParameterValues("ECM_Example")
parameter_values.update(
    {
        "Cell capacity [A.h]": 3,
        "Nominal cell capacity [A.h]": 3,
        "Element-1 initial overpotential [V]": 0,
        "Upper voltage cut-off [V]": 4.2,
        "Lower voltage cut-off [V]": 2.5,
        "R0 [Ohm]": 1e-3,
        "R1 [Ohm]": 3e-3,
        "C1 [F]": 5e2,
    }
)
# Optional arguments - only needed for two RC pairs
parameter_values.update(
    {
        "R2 [Ohm]": 2e-3,
        "C2 [F]": 3e4,
        "Element-2 initial overpotential [V]": 0,
    },
    check_already_exists=False,
)

We use multiple HPPC pulses from the dataset: Kollmeyer, Phillip; Skells, Michael (2020), “Samsung INR21700 30T 3Ah Li-ion Battery Data”, Mendeley Data, V1, doi: 10.17632/9xyvy2njj3.1 

In [ ]:
file_loc = r"../../data/Samsung_INR21700/multipulse_hppc.xlsx"
df = pd.read_excel(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(),
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)

In [ ]:
r0_guess = 0.005
parameter_values.update(
    {
        "R0 [Ohm]": pybop.Parameter(
            prior=pybop.Gaussian(r0_guess, r0_guess / 10),
            bounds=[0, 0.1],
        ),
        "R1 [Ohm]": pybop.Parameter(
            prior=pybop.Gaussian(r0_guess, r0_guess / 10),
            bounds=[0, 0.1],
        ),
        "C1 [F]": pybop.Parameter(
            prior=pybop.Gaussian(500, 100),
            bounds=[100, 1000],
        ),
        "R2 [Ohm]": pybop.Parameter(
            prior=pybop.Gaussian(r0_guess, r0_guess / 10),
            bounds=[0, 0.1],
        ),
        "C2 [F]": pybop.Parameter(
            prior=pybop.Gaussian(2000, 500),
            bounds=[1000, 10000],
        ),
    }
)

In [ ]:
# To see current vs time profile.
fig1 = go.Figure()
# Add a line trace for current vs. time
fig1.add_trace(
    go.Scatter(
        x=df["Time"].to_numpy(),
        y=df["Current"].to_numpy(),
        mode="lines",  # 'lines', 'markers', or 'lines+markers'
        name="Current vs Time",
    )
)

# Customize layout
fig1.update_layout(
    title="Current vs Time",
    xaxis_title="Time (s)",
    yaxis_title="Current (A)",
    template="plotly",  # Use a Plotly template (optional)
)

# Show the plot
fig1.show()

In [ ]:
# To see voltage vs time profile.
fig2 = go.Figure()
# Add a line trace for current vs. time
fig2.add_trace(
    go.Scatter(
        x=df["Time"].to_numpy(),
        y=df["Voltage"].to_numpy(),
        mode="lines",  # 'lines', 'markers', or 'lines+markers'
        name="Voltage vs Time",
    )
)

# Customize layout
fig2.update_layout(
    title="Voltage vs Time",
    xaxis_title="Time (s)",
    yaxis_title="Voltage (V)",
    template="plotly",  # Use a Plotly template (optional)
)

# Show the plot
fig2.show()

The `Problem` class provides us with a single class that holds all of the objects we need to evaluate our selected `SumSquaredError` cost function.

Initial state can be either "Initial SoC" or "Initial open-circuit voltage [V]". In this example, we get the initial OCV by accessing the voltage data. However, user can simply use a value instead, e.g., {"Initial open-circuit voltage [V]": 4.1}. Similarly, if SOC input is required, {"Initial SoC": 0.95}.

In [ ]:
parameter_values.set_initial_state(f"{df['Voltage'].to_numpy()[0]} V")
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=dataset,
    solver=pybamm.CasadiSolver(mode="safe", dt_max=40),
)
cost = pybop.SumSquaredError(dataset)
problem = pybop.Problem(simulator, cost)

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the CMA-ES method as it provides global optimisation capability. For the sake of reducing the runtime of this example, we limit the maximum iterations to 100; however, feel free to update this value.

In [ ]:
options = pybop.PintsOptions(
    max_unchanged_iterations=20,
    max_iterations=100,
)
optim = pybop.XNES(problem, options=options)
result = optim.run()

## Plotting and Visualisation

PyBOP provides various plotting utilities to visualize the results of the optimisation.

In [ ]:
pybop.plot.problem(problem, problem_inputs=result.x, title="Optimised Comparison");

### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
result.plot_convergence()
result.plot_parameters();

### Conclusion

This notebook illustrates how to perform parameter estimation for multi-pulse HPPC data, providing insights into the optimisation process through various visualisations.